In [1]:
!pip install -q findspark

import findspark
findspark.init()

In [4]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]").getOrCreate()

In [151]:
# Reading Dataframe with given schema:
from pyspark.sql.types import StructType,StructField, StringType, IntegerType, DoubleType
def get_schema(d):
    new_schema = StructType()
    for key, value in d.items():
        new_schema.add(key,value,True)
    return new_schema

d = {'year':IntegerType(),'imdb':StringType(),'title':StringType(),'test':StringType(),'clean_test':StringType(),
    'binary':StringType(),'budget':DoubleType(),'domgross':StringType(),'intgross':StringType(),'code':StringType(),
    'budget_2013$':DoubleType(),'domgross_2013$':StringType(),'intgross_2013$':StringType(),'period code':IntegerType(),
    'decade code':IntegerType()}
df=spark.read.csv("movies.csv",header=True,schema=get_schema(d))
df = df.toDF(*[col.replace(' ','_').replace(':','').replace('-','_').replace('$','') for col in df.columns])

In [156]:
# Preparing init Data:
# INFO : Primary key is "imdb" column
init = df.select(df.columns[:5]).limit(5000)
init.count()

1794

In [157]:
# Preparing a delta data:
delta = init.limit(3)
delta = delta.replace('tt1711425','aa0000000')
delta = delta.replace('ok-disagree','updated_val')
delta.show()

+----+---------+----------------+---------------+----------+
|year|     imdb|           title|           test|clean_test|
+----+---------+----------------+---------------+----------+
|2013|aa0000000|   21 &amp; Over|         notalk|    notalk|
|2012|tt1343727|        Dredd 3D|    updated_val|        ok|
|2013|tt2024544|12 Years a Slave|notalk-disagree|    notalk|
+----+---------+----------------+---------------+----------+



# Create SCD Dataset

In [163]:
delta.createOrReplaceTempView('delta')
init.createOrReplaceTempView('init')

In [170]:
# Primary Key check:
p = ['imdb','year']
p = ','.join(p)
q = "select "+p+" from init group by "+p+" having count(*)>1"
spark.sql(q).count()

0

In [172]:
q = "select * from init where concat("+p+") not in (select concat("+p+") from delta) union select * from delta"
merge = spark.sql(q)
merge.count()

1795